In [ ]:
import requests
import json
import urllib
import datetime
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from google.colab import drive
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
drive.mount('/content/gdrive')

# Грузим данные

In [ ]:
#deribit
start_time = datetime.datetime(2021,7,6,0,0,0)
start_time = str(int(start_time.timestamp())*1000)
end_time = datetime.datetime(2021,7,10,0,0,0)
end_time = str(int(end_time.timestamp())*1000)
has_more = True
trades = {'price':[],'timestamp':[],'time':[]}
while has_more == True:
  request = 'https://deribit.com/api/v2/public/get_last_trades_by_instrument_and_time?count=1000&start_timestamp='+start_time+'&end_timestamp='+end_time+'&instrument_name=BTC-24SEP21'
  response = urllib.request.urlopen(request)
  result = response.read()
  has_more = json.loads(result)['result']['has_more']
  print(has_more)
  result = json.loads(result)['result']['trades']
  for i in result:
    trades['price'].append(i['price'])
    trades['timestamp'].append(i['timestamp'])
    trades['time'].append(pd.to_datetime(datetime.datetime.fromtimestamp(i['timestamp']/1000).strftime('%Y-%m-%d %H:%M:%S')))
  start_time = str(np.max(trades['timestamp']))
DERIBIT_trades = pd.DataFrame(trades)

In [ ]:
#binance

raw_data_0507 = pd.read_csv("/content/gdrive/My Drive/Quant/data/binance trades/BTCUSDT_210924-trades-2021-07-05.csv", header = None)
raw_data_0607 = pd.read_csv("/content/gdrive/My Drive/Quant/data/binance trades/BTCUSDT_210924-trades-2021-07-06.csv", header = None)
raw_data_0707 = pd.read_csv("/content/gdrive/My Drive/Quant/data/binance trades/BTCUSDT_210924-trades-2021-07-07.csv", header = None)
raw_data_0807 = pd.read_csv("/content/gdrive/My Drive/Quant/data/binance trades/BTCUSDT_210924-trades-2021-07-08.csv", header = None)
raw_data_0907 = pd.read_csv("/content/gdrive/My Drive/Quant/data/binance trades/BTCUSDT_210924-trades-2021-07-09.csv", header = None)
BINANCE_trades = pd.concat([raw_data_0507,raw_data_0607, raw_data_0707, raw_data_0807,raw_data_0907])[[1,4]]
BINANCE_trades.columns = ['price','timestamp']
time = []
for i in range(0,len(BINANCE_trades)):
  time.append(pd.to_datetime(datetime.datetime.fromtimestamp(BINANCE_trades['timestamp'].iloc[i]/1000).strftime('%Y-%m-%d %H:%M:%S')))
BINANCE_trades['time']=time

In [ ]:
minutes = pd.date_range(start = '2021-07-03 00:00:00',end = '2021-07-05 00:00:00',freq = '1min')
changes = {'time':[],'open':[],'close':[],'high':[],'low':[]}
counter = 0
for i in range(len(minutes)-1):
  bsml = BINANCE_trades[(BINANCE_trades['time']>=minutes[i])&(BINANCE_trades['time']<minutes[i+1])]
  dsml = DERIBIT_trades[(DERIBIT_trades['time']>=minutes[i])&(DERIBIT_trades['time']<minutes[i+1])]
  j = 0
  k = 0
  if (len(dsml)==0 or len(bsml)==0):
    counter+=1
    continue
  last_time_bsml = bsml['timestamp'].iloc[0]
  last_time_dsml = dsml['timestamp'].iloc[0]

  changes['open'].append(dsml['price'].iloc[0] - bsml['price'].iloc[0])
  changes['time'].append(minutes[i])
  changes['close'].append(dsml['price'].iloc[-1] - bsml['price'].iloc[-1])
  
  lilchan = []
  bsml['type'] = [-1 for i in range(len(bsml))]
  dsml['type'] = [1 for i in range(len(dsml))]
  concated = pd.concat([bsml,dsml])
  concated = concated.sort_values(by='timestamp')
  k = 0
  while concated['type'].iloc[k]==1:
    k+=1
  j = 0
  while concated['type'].iloc[j]==-1:
    j+=1
  j=max(j,k)
  last_a = concated.iloc[j]
  last_b = concated.iloc[j-1]
  lilchan.append(concated['price'].iloc[j]*concated['type'].iloc[j]+concated['price'].iloc[j-1]*concated['type'].iloc[j-1])
  j+=1
  
  while(j<len(concated)):
    if last_a['type'] == concated['type'].iloc[j]:
      lilchan.append(lilchan[-1]-last_a['price']*last_a['type']+concated['price'].iloc[j]*concated['type'].iloc[j])
      last_a = concated.iloc[j]
    else:
      lilchan.append(lilchan[-1]-last_b['price']*last_b['type']+concated['price'].iloc[j]*concated['type'].iloc[j])
      last_b = concated.iloc[j]
    #print(last_a['price'], last_b['price'],lilchan[-1])
    j+=1
  #lilchan = []
  a = concated[concated['type']==-1]
  b = concated[concated['type']==1] 
  changes['high'].append(max(lilchan))
  changes['low'].append(min(lilchan))
changes = pd.DataFrame(changes)

# Смотрим характеристики спреда

In [ ]:
beginning = pd.to_datetime('2021-07-06 00:00:00')
ending =   pd.to_datetime('2021-07-10 00:00:00')
feerate = 0.001
changes = {'timestamp':[], 'time':[], 'price':[], 'fee':[], 'prper':[]}
counter = 0
for i in range(0,1):#len(minutes)-1):
  bsml = BINANCE_trades[(BINANCE_trades['time']>=beginning)&(BINANCE_trades['time']<ending)]
  dsml = DERIBIT_trades[(DERIBIT_trades['time']>=beginning)&(DERIBIT_trades['time']<ending)]
  j = 0
  k = 0
  if (len(dsml)==0 or len(bsml)==0):
    counter+=1
    continue

  bsml['type'] = [-1 for i in range(len(bsml))]
  dsml['type'] = [1 for i in range(len(dsml))]
  concated = pd.concat([bsml,dsml])
  concated = concated.sort_values(by='timestamp')
  k = 0
  while concated['type'].iloc[k]==1:
    k+=1
  j = 0
  while concated['type'].iloc[j]==-1:
    j+=1
  j=max(j,k)

  last_a = concated.iloc[j]
  last_b = concated.iloc[j-1]

  changes['price'].append(concated['price'].iloc[j]*concated['type'].iloc[j]+concated['price'].iloc[j-1]*concated['type'].iloc[j-1])
  changes['time'].append(pd.to_datetime(concated['time'].iloc[j]))
  changes['timestamp'].append(concated['timestamp'].iloc[j])
  changes['fee'].append((concated['price'].iloc[j]+concated['price'].iloc[j-1])*feerate)
  changes['prper'].append(concated['price'].iloc[j]+concated['price'].iloc[j-1])
  j+=1
  
  while(j<len(concated)):
    if last_a['type'] == concated['type'].iloc[j]:
      changes['price'].append(changes['price'][-1]-last_a['price']*last_a['type']+concated['price'].iloc[j]*concated['type'].iloc[j])
      last_a = concated.iloc[j]
      changes['time'].append(pd.to_datetime(last_a['time']))
      changes['timestamp'].append(last_a['timestamp'])
      changes['fee'].append((last_a['price']+last_b['price'])*feerate)
      changes['prper'].append(last_a['price']+last_b['price'])
    else:
      changes['price'].append(changes['price'][-1]-last_b['price']*last_b['type']+concated['price'].iloc[j]*concated['type'].iloc[j])
      last_b = concated.iloc[j]
      changes['time'].append(pd.to_datetime(last_b['time']))
      changes['timestamp'].append(last_b['timestamp'])
      changes['fee'].append((last_a['price']+last_b['price'])*feerate)
      changes['prper'].append(last_a['price']+last_b['price'])
    #print(j)
    j+=1

changes = pd.DataFrame(changes)

In [ ]:
plt.figure(figsize = (10,10))
plt.plot(range((len(changes))),changes['price']/changes['fee'], color ='red')
plt.grid()
plt.title('Спреды по уровням в зависимости от времени')

In [ ]:
#квантили
hours = pd.date_range(start = '2021-07-06 00:00:00',end = '2021-07-10 00:00:00',freq = '15min')
results = {'mean':[],'std':[],'minmax':[]}
for i in range(len(hours[:-1])):
  lch = changes[(changes['time']>=hours[i])&((changes['time']<hours[i+1]))]
  results['mean'].append(np.mean(lch['price']/lch['fee']))
  results['std'].append(np.std(lch['price']/lch['fee']))
  results['minmax'].append((np.max(lch['price']/lch['fee'])-np.min(lch['price']/lch['fee'])))
results = pd.DataFrame(results)
#results = np.array(results['mean'].iloc[1:].to_list())-np.array(results['mean'].iloc[:-1].to_list())
step = 0.05
print(np.max(results['minmax']),np.std(results['minmax']))
print(' q \t   v')
for i in np.arange(0,1,step):
  print(np.around(i+step,2),'\t',np.around(np.quantile(np.array(results['mean'].iloc[1:].to_list())-np.array(results['mean'].iloc[0:-1].to_list()),i+step),2))

In [ ]:
#квантили
hours = pd.date_range(start = '2021-07-06 00:00:00',end = '2021-07-10 00:00:00',freq = '15min')
results = {'mean':[],'std':[],'minmax':[]}
for i in range(len(hours[:-1])):
  lch = changes[(changes['time']>=hours[i])&((changes['time']<hours[i+1]))]
  results['mean'].append(np.mean(lch['price']))
  results['std'].append(np.std(lch['price']))
  results['minmax'].append((np.max(lch['price'])-np.min(lch['price'])))
results = pd.DataFrame(results)
#results = np.array(results['mean'].iloc[1:].to_list())-np.array(results['mean'].iloc[:-1].to_list())
step = 0.05
print(np.max(results['minmax']),np.std(results['minmax']))
print(' q \t   v')
for i in np.arange(0,1,step):
  print(np.around(i+step,2),'\t',np.around(np.quantile(np.array(results['mean'].iloc[1:].to_list())-np.array(results['mean'].iloc[0:-1].to_list()),i+step),2))

In [ ]:
fees = []
step = 0.25
for i in np.arange(-4,4,step):
  fees.append(len((changes[(changes['price']<changes['fee']*(i+step)) & (changes['price'] >= changes['fee']*i)]))/len(changes))
plt.figure(figsize = (10,10))
plt.plot(np.arange(-4,4,step),fees)
plt.grid()
plt.title('Распределение спреда в зависимости от доли от сборов')


In [ ]:
#абсолютные квантили
print(np.mean(changes['price']/changes['fee']))
print(np.std(changes['price']/changes['fee']))
step = 0.05
print(' q \t   v')
levels = {'квантиль':[], 'цена/сбор':[], 'цена абс.':[],'цена отн.':[]}
for i in np.arange(0,1,step):
  print(np.around(i+step,2),'\t',np.around(np.quantile(changes['price']/changes['fee'],i+step),2))
  levels['квантиль'].append(i+step)
  levels['цена/сбор'].append(np.around(np.quantile(changes['price']/changes['fee'],i+step)))
#print(' ')
for i in np.arange(0,1,step):
  print(np.around(i+step,2),'\t',np.around(np.quantile(changes['price'],i+step),2))
  levels['цена абс.'].append(np.around(np.quantile(changes['price'],i+step),2))
#print(' ')
for i in np.arange(0,1,step):
  print(np.around(i+step,2),'\t',np.around(np.quantile(changes['price']/changes['prper'],i+step),5))
  levels['цена отн.'].append(np.around(np.quantile(changes['price']/changes['prper'],i+step),5))

In [ ]:
hours = pd.date_range(start = '2021-07-06 00:00:00',end = '2021-07-10 00:00:00',freq = '1H')
levels['1H абс.'] = []
#levels['1H относ.'] = []
for j in range(len(levels['цена абс.'])):
  ka = []
  kr = []
  for t in range(len(hours)-1):
    lch = changes[(changes['time']>=hours[t])&((changes['time']<hours[t+1]))]
    ka.append(len(lch[lch['price']<=levels['цена абс.'][j]]))
    kr.append(ka[-1]/len(lch))
  levels['1H абс.'].append(np.around(np.mean(ka)))
  #levels['1H относ.'].append(np.around(np.mean(kr),4))

In [ ]:
levels['1H абс.']

In [ ]:
#тест Дики-Фуллера
from statsmodels.tsa.stattools import adfuller
adresults = []
hours = pd.date_range(start = '2021-07-05 00:00:00',end = '2021-07-09 00:00:00',freq = '5h')

for i in range(len(hours)-1):
  a = changes[(changes['time']>=hours[i]) & (changes['time']<hours[i+1])]
  if len(a)>0:
   adresults.append(adfuller(a['price']/a['fee'], regression='c')[0])

In [ ]:
np.sum([i<-3.5 for i in adresults])/len(adresults)

In [ ]:
periods = [300, 600, 1200, 2400]
thresholds = [1,1.2,1.4,1.6,1.8,2.0]
for threshold in thresholds:
  print('\nпороговое значение:',threshold)
  #print('период ск.средней', 'количество сделок', 'ср. профит', 'мин.профит', 'ср. время удержания')
  for period in periods:
    upper = []
    lower = []
    times = []
    memento = 0
    upperflag = False
    lowerflag = False
    consider = pd.DataFrame(changes['price']/changes['fee'])
    consider['time'] = changes['time']
    consider.columns = ['price','time']
    sma = consider['price'].rolling(period).mean()
    for i in range(period, len(consider)):
      #print(consider.iloc[i]-sma.iloc[i])
      if upperflag == False and consider['price'].iloc[i]-sma.iloc[i]>threshold:
        tbeg = consider['time'].iloc[i]
        #memento = consider['price'].iloc[i]
        memento = sma.iloc[i]+threshold
        upperflag = True
      elif upperflag == True and consider['price'].iloc[i]-sma.iloc[i]<0:
        times.append(consider['time'].iloc[i]-tbeg)
        #upper.append(memento - consider['price'].iloc[i])
        upper.append(memento - sma.iloc[i])
        upperflag = False
    print(period,'\t', len(upper),'\t', np.around(np.mean(upper),3),'\t', np.around(np.min(upper),3), '\t', np.mean(times))

In [ ]:
periods = [300, 600, 1200, 2400]
thresholds = [1,1.2,1.4,1.6,1.8,2.0]
for threshold in thresholds:
  print('\nпороговое значение:',threshold)
  #print('период ск.средней', 'количество сделок', 'ср. профит', 'мин.профит', 'ср. время удержания')
  for period in periods:
    upper = []
    lower = []
    times = []
    memento = 0
    upperflag = False
    lowerflag = False
    consider = pd.DataFrame(changes['price']/changes['fee'])
    consider['time'] = changes['time']
    consider.columns = ['price','time']
    sma = consider['price'].rolling(period).mean()
    for i in range(period, len(consider)):
      #print(consider.iloc[i]-sma.iloc[i])
      if upperflag == False and consider['price'].iloc[i]-sma.iloc[i]<-threshold:
        tbeg = consider['time'].iloc[i]
        #memento = consider['price'].iloc[i]
        memento = sma.iloc[i] - threshold
        upperflag = True
      elif upperflag == True and consider['price'].iloc[i]-sma.iloc[i]>0:
        times.append(consider['time'].iloc[i]-tbeg)
        #upper.append(-memento + consider['price'].iloc[i])
        upper.append(-memento + sma.iloc[i])
        upperflag = False
    print(period,'\t', len(upper),'\t', np.around(np.mean(upper),3),'\t', np.around(np.min(upper),3),'\t', np.mean(times))

In [ ]:
print(np.mean(upper), np.min(upper),len(upper), np.mean(times))

In [ ]:
np.mean(times)

# Трейдингвью, но спред считаем сами


In [ ]:
BINANCE_candles = pd.read_csv("/content/gdrive/My Drive/Quant/data/tradingview/BINANCE_BTCU2021, 1 (3).csv")
DERIBIT_candles = pd.read_csv("/content/gdrive/My Drive/Quant/data/tradingview/DERIBIT_BTC24U21, 1 (1).csv")
dat = []
for i in range(0,len(DERIBIT_candles)):
  dat.append(pd.to_datetime(datetime.datetime.fromtimestamp(DERIBIT_candles['time'].iloc[i]).strftime('%Y-%m-%d %H:%M:%S')))
DERIBIT_candles['datetime']=dat
dat = []
for i in range(0,len(BINANCE_candles)):
  dat.append(pd.to_datetime(datetime.datetime.fromtimestamp(BINANCE_candles['time'].iloc[i]).strftime('%Y-%m-%d %H:%M:%S')))
BINANCE_candles['datetime']=dat

In [ ]:
changes_high = []
changes_low = []
merged = pd.merge(DERIBIT_candles, BINANCE_candles, on = ['datetime'], suffixes = ['DER','BIN'])
merged = merged[merged['datetime']>pd.to_datetime("2021-06-27")]
for i in range(len(merged)):
  changes_high.append((merged['highDER'].iloc[i]-merged['highBIN'].iloc[i])/(merged['highDER'].iloc[i]*0.001+merged['highBIN'].iloc[i]*0.001))
  changes_low.append((merged['lowDER'].iloc[i]-merged['lowBIN'].iloc[i])/(merged['lowDER'].iloc[i]*0.001+merged['lowBIN'].iloc[i]*0.001))
  #changes.append(merged['open_DER'].iloc[i]-merged['close_BIN'].iloc[i])

In [ ]:
plt.figure(figsize = (10,10))
plt.plot(merged['datetime'],changes_high)
plt.grid()
#plt.plot(range(len(changes_low)),changes_low)

In [ ]:
num_mins = len(merged)
results = {'mean':[],'std':[]}
for t in range(0,num_mins,30):
  d = []
  for j in range(30):
    try:
      d.append((changes_high[j+t]+changes_low[j+t])/2)
    except:
      break
  results['mean'].append(np.mean(d))
  results['std'].append(np.std(d))
 # results.append((np.mean(d), np.std(d)))
results = pd.DataFrame(results)

In [ ]:
changes_high = []
changes_low = []
for i in range(len(merged)):
  changes_high.append((merged['highDER'].iloc[i]-merged['highBIN'].iloc[i]))
  if changes_high[-1]==0:
    changes_high[-1]=0.001
  changes_low.append((merged['lowDER'].iloc[i]-merged['lowBIN'].iloc[i]))
deltas1 = (-np.array(changes_high[0:-2])+np.array(changes_high[1:-1]))/np.array(changes_high[0:-2])
deltas2 = (-np.array(changes_high[1:-1])+np.array(changes_high[2:]))/np.array(changes_high[1:-1])
np.corrcoef(deltas1,deltas2)

In [ ]:
for i in changes_high:
  if i==0:
    print(i)

In [ ]:
step = 0.05
print(' q \t   v')
for i in np.arange(0,1,step):
  print(np.around(i+step,2),'\t',np.around(np.quantile(np.array(results['mean'].iloc[1:].to_list())-np.array(results['mean'].iloc[0:-1].to_list()),i+step),2))


In [ ]:
np.array(results['mean'].iloc[1:].to_list())-np.array(results['mean'].iloc[0:-1].to_list())

In [ ]:
results['mean'].iloc[0:-1]

# Трейдингвью, но спред

In [ ]:
SPREAD_candles = pd.read_csv("/content/gdrive/My Drive/Quant/data/tradingview/HUOBI_ETH24U2021-KRAKEN_ETHUSD24U, 1.csv")
dat = []
for i in range(0,len(SPREAD_candles)):
  dat.append(pd.to_datetime(datetime.datetime.fromtimestamp(SPREAD_candles['time'].iloc[i]).strftime('%Y-%m-%d %H:%M:%S')))
SPREAD_candles['datetime']=dat
SPREAD_candles[SPREAD_candles['datetime']>pd.to_datetime('2021-07-01')]

In [ ]:
a = SPREAD_candles[['time','datetime','high']]
b = SPREAD_candles[['time','datetime','low']]
c = SPREAD_candles[['time','datetime','open']]
d = SPREAD_candles[['time','datetime','close']]
a.columns = ['time','datetime','price']
b.columns = ['time','datetime','price']
c.columns = ['time','datetime','price']
d.columns = ['time','datetime','price']
candles = pd.concat([a,b,c,d,])
candles.sort_values('time',inplace = True)

In [ ]:
candles['rolling'] = candles['price'].rolling(4260).mean()
candles['ewm'] = (candles['price'].ewm(alpha = 0.001).mean())

In [ ]:
plt.plot(range(len(candles['datetime'])),candles['price'])
plt.plot(range(len(candles['datetime'])),candles['ewm'])
from statsmodels.tsa.stattools import adfuller
adfuller(candles['price'],1)

In [ ]:
#уровни
step = 0.05
print(' q \t   v')
levels = []
quantiles = []
for i in np.arange(0,1,step):
  quantiles.append(np.around(i+step,2))
  print(np.around(i+step,2),'\t',np.around(np.quantile(candles['price'],i+step),4))
  levels.append(np.quantile(candles['price'],i+step))

In [ ]:
hours = pd.date_range(start = '2021-07-01 00:00:00',end = '2021-07-10 00:00:00',freq = '1h')
levels1h = []
delta1h = []
for j in levels:
  k = []
  for i in range(len(hours)-1):
    a = candles[(candles['datetime']>=hours[i]) & (candles['datetime']<hours[i+1])]
    k.append(len(a[a['price']<=j]))
  levels1h.append(np.mean(k))

In [ ]:
hours = pd.date_range(start = '2021-07-01 00:00:00',end = '2021-07-10 00:00:00',freq = '24h')
levels24h = []
delta24h = []
for j in levels:
  k = []
  for i in range(len(hours)-1):
    a = candles[(candles['datetime']>=hours[i]) & (candles['datetime']<hours[i+1])]
    k.append(len(a[a['price']<=j]))
  levels24h.append(np.around(np.mean(k),2))

In [ ]:
hours = pd.date_range(start = '2021-07-01 00:00:00',end = '2021-07-10 00:00:00',freq = '168h')
levels1w = []
delta1w = []
for j in levels:
  k = []
  c = []
  for i in range(len(hours)-1):
    a = candles[(candles['datetime']>=hours[i]) & (candles['datetime']<hours[i+1])]
    k.append(len(a[a['price']<=j]))
  levels1w.append(np.around(np.mean(k),2))


In [ ]:
pd.DataFrame({'prob':quantiles, 'levels':levels, 'average 1h':levels1h, 'average 24h':levels24h, 'average 168h':levels1w})